In [ ]:
import os

if __name__ == "__main__":  # 이 파일이 직접 실행될 때만
    try:
        from google.colab import drive
        drive.mount('/content/drive')
    except:
        pass

In [ ]:
import os
import re
import json
import cv2
import numpy as np
import unicodedata
import easyocr
from openai import OpenAI
from PIL import Image
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import base64

# ✅ 설정
reader = easyocr.Reader(['ko', 'en'], gpu=True)
client = OpenAI(api_key=os.environ.get(" "))

# 1. 모델 로딩
model = SentenceTransformer("BM-K/KoSimCSE-roberta-multitask")
models = {"kosimcse": model}  # ✅ 여기에 추가!

# ✅ OCR 함수
def extract_text_with_underlines(image_path):
    image = Image.open(image_path).convert("RGB")
    img_np = np.array(image)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    results = reader.readtext(gray, detail=True)
    full_text = " ".join([text for (_, text, _) in results])
    return re.sub(r'\b1[\.\)]', '①', full_text)\
             .replace('2.', '②')\
             .replace('3.', '③')\
             .replace('4.', '④')\
             .replace('5.', '⑤')\
             .strip()

# ✅ GPT 프롬프트 (수정 금지)
text_prompt = '''
다음은 국어 언어와 매체 문제입니다. 지문과 문제(선택지 포함)를 꼼꼼히 읽고 다음을 수행하세요:

1. 질문 조건을 정확히 반영해 정답을 선택하세요.
2. 반드시 ①~⑤ 중 하나만 골라 [정답] ③ 형식으로 답하세요.
3. 지문에 근거한 해설을 [해설]로 3~5문장 쓰세요.

아래 이미지는 문학 문제 하나의 '질문 문장 + ①~⑤ 선택지'가 포함된 이미지야.
    만약 <보기> 문장이 존재한다면 질문 앞에 위치하며, 반드시 포함해서 출력해.

    형식은 아래와 같이 출력해:

    (질문과 <보기> 내용. <보기>가 없다면 생략)
    ① ...
    ② ...
    ③ ...
    ④ ...
    ⑤ ...

    ❗ 절대 설명이나 부가 텍스트를 추가하지 말고 형식 그대로 출력해.

[지문]
{passage}

[문제]
{question}

결과는 다음 형식으로 출력하세요:

[정답] ③
[해설] … (여기에 근거 설명)

'''

# ✅ GPT 실행 및 파싱
def ask_gpt(prompt_text):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0.3
    )
    return resp.choices[0].message.content

def parse_gpt_output(output):
    a = re.search(r'\[정답\]\s*([①-⑤])', output)
    e = re.search(r'\[해설\](.*)', output, re.DOTALL)
    return (a.group(1) if a else None), (e.group(1).strip() if e else "")

In [ ]:
# 파일 이름 정렬
input_dir = "/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/수능 국어 AI 튜터링 시스템/언매/data/save_json_tagged"
file_list = sorted([f for f in os.listdir(input_dir) if f.endswith(".json")])

# 1. 모델과 데이터 준비
print("모델과 데이터 불러오는 중...")
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

data = []
for file_name in file_list:
    file_path = os.path.join(input_dir, file_name)
    with open(file_path, encoding="utf-8") as f:
        problems = json.load(f)
        data.extend(problems)

passages = [item["지문"] for item in data]
embeddings = model.encode(passages, convert_to_tensor=False)


MEDIA_RELATED_TAGS = {
    "정보 제시 이해", "매체 구성 요소 분석", "매체 표현 방식 이해", "매체의 목적 분석",
    "매체의 효과 분석", "매체의 시각적 요소 이해", "매체의 청각적 요소 이해",
    "매체의 상징적 의미 이해", "매체의 사회적 맥락 이해", "대화 내용 반영",
    "비판적 사고", "자료 분석", "적절하지 않은 것 찾기", "문장 요소 분석",
    "게시판 구성 이해", "인터뷰 분석", "시각 정보 해석", "그래프 이해", "도표 분석"
}

def get_problem_type(tag_list):
    return "매체" if any(tag in MEDIA_RELATED_TAGS for tag in tag_list) else "언어"


for file_name in file_list:
    full_path = os.path.join(input_dir, file_name)
    with open(full_path, "r", encoding="utf-8") as f:
        problems = json.load(f)

    for prob in problems:
        # tags 필드를 문자열에서 리스트로 변환 (쉼표 기준)
        tags = prob.get("tags", [])
        if isinstance(tags, str):
            tags = [tag.strip() for tag in tags.split(",") if tag.strip()]
        prob["tags"] = tags

        # 유형 추가
        prob["type"] = get_problem_type(tags)



'''
def normalize_filename(name):
    # 유니코드 정규화, 공백/특수문자/언더스코어/하이픈 제거, 소문자 변환
    name = unicodedata.normalize('NFC', name)
    name = name.replace(' ', '').replace('_', '').replace('-', '').lower()
    return name
'''

def extract_number_from_question(qtext):
    match = re.match(r"^\s*(\d+)\.", qtext)
    return match.group(1) if match else ""


def recommend_for_external_problem(target_problem, top_n=3):
    target_tags = target_problem.get("tags", [])
    target_kw_embedding = model.encode(" ".join(target_tags), convert_to_tensor=False)

    target_passage = target_problem.get("지문", "")
    has_target_passage = bool(target_passage.strip())
    if has_target_passage:
        target_embedding = model.encode(target_passage, convert_to_tensor=False)

    # 1. 문제 유형 판별 (매체 포함 여부)
    target_type = "매체" if any("매체" in tag for tag in target_tags) else "언어"

    results = []
    for item in data:
        item_tags = item.get("tags", [])
        item_passage = item.get("지문", "")
        has_item_passage = bool(item_passage.strip())

        if not item_tags:
            continue  # 태그가 없으면 제외

        # 2. 유형이 다르면 스킵
        item_type = "매체" if any("매체" in tag for tag in item_tags) else "언어"
        if item_type != target_type:
            continue

        # 태그 유사도
        item_kw_embedding = model.encode(" ".join(item_tags), convert_to_tensor=False)
        keyword_sim = cosine_similarity([target_kw_embedding], [item_kw_embedding])[0][0]

        # 지문 유사도
        if has_item_passage:
            item_embedding = model.encode(item_passage, convert_to_tensor=False)
            passage_sim = cosine_similarity([target_embedding], [item_embedding])[0][0]
            final_score = 0.5 * passage_sim + 0.5 * keyword_sim
        else:
            passage_sim = None
            final_score = keyword_sim

        results.append({
            "year": item.get("년", ""),
            "month": item.get("월", ""),
            "문제" : item.get("문제", ""),
            "id_str": f"{item.get('년', '??')}학년도 {item.get('월', '??')} 지문 {extract_number_from_question(item.get('문제', ''))}번",
            "score": round(final_score, 4),
            "embedding_sim": round(passage_sim, 4) if passage_sim is not None else None,
            "keyword_cosine": round(keyword_sim, 4),
            "preview": item_passage[:100]
        })

    return sorted(results, key=lambda x: x["score"], reverse=True)[:top_n]




def tag_from_image(image_path):
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 국어 문제 이미지 분석 전문가야."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text":
                     """
                        아래는 국어 영역 언어와 매체 문제입니다.
                        유형이 '언어' 인 경우, 언어 관련 태그를, '매체'인 경우 매체 관련 태그를 생성해 주세요.

                        유형이 '언어'인 경우에는, 언어 관련 태그를 5개 생성하고,
                        유형이 '매체'인 경우에는, 매체 관련 태그를 3개, 문제의 출제 의도 태그를 2개씩 생성하여 총 5개의 태그를 생성해주세요.
                        태그는 쉼표로 구분해서 출력해 주세요.

                        언어 관련 태그의 후보에는 '훈민정음 이해', '중세 국어 변화', '단어 변화 양상', '모음 조화 이해', '시간 표현 이해', '부사어', '조사어', '서술어 자릿수', '사잇소리 표기 분석', '동음이의어 분석', '다의어 분석',
                        '자음 분석', '음운의 변동', '간접 인용', '종결 어미', '발화 형식 비교', '관형사절', '시간적 관계 이해', '사동 및 피동 표현', '주동 및 능동 표현', '문장 성분 분석' 등이 있고,

                        위에서 언급한 언어 관련 태그도 예시일 뿐, 이 중에서만 찾으려고 하지말고, 다른 태그의 후보도 적극적으로 다양하게 찾으세요.

                        매체 관련 태그의 후보에는 '정보 제시 이해', '매체 구성 요소 분석', '매체 표현 방식 이해', '매체의 목적 분석', '매체의 효과 분석', '매체의 시각적 요소 이해', '매체의 청각적 요소 이해', '매체의 상징적 의미 이해', '매체의 사회적 맥락 이해' 등이 있습니다.
                        그리고 매체 관련 태그의 출제 의도는 '매체 구성 이해', '대화 내용 반영', '정보 제공 방식 분석', '비판적 사고', '자료 분석', '문장 요소 분석', '적절하지 않은 것 찾기' 등이 있습니다.

                        매체 관련 태그의 후보, 출제 의도도 위에서 적어준 것은 예시일 뿐, 절대 이 중에서만 찾으려고 하지말고, 적극적으로 다양하게 찾으세요.
                        언어 관련한 문제 출력 예시) 안긴문장 분석, 서술어 자릿수, 의미 관계 파악
                        매체 관련한 문제 출력 예시) 게시판 구성 이해, 대화 내용 반영, 정보 제공 방식 분석, 비판적 사고, 자료 분석
                            {text}
                     """},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        temperature=0.5
    )

    content = response.choices[0].message.content.strip()
    print("GPT Vision 응답 원본:\n", content)

    # 태그를 리스트로 분리
    tag_list = [tag.strip() for tag in content.split(",") if tag.strip()]

    # 유형 판별: 매체 관련 태그 중 하나라도 포함되면 매체
    problem_type = "매체" if any(tag in MEDIA_RELATED_TAGS for tag in tag_list) else "언어"

    return {
        "tags": tag_list,
        "type": problem_type,
        "지문": "[이미지 기반 지문 생략]",
    }

# ✅ 매핑
def get_passage_mapping(year, month):
    # 예외 정의
    p9_range = {
        (2022, "03"): range(38, 40),  # 38~39
    }

    p10_range = {
        (2022, "09"): range(40, 43),
        (2024, "09"): range(40, 43),
    }

    # 기본값
    p9_default = range(35, 37)
    p10_default = range(40, 44)

    y, m = year, month

    mapping = {}

    # p9
    p9 = p9_range.get((y, m), p9_default)
    for i in p9:
        mapping[i] = "p9"

    # p10
    p10 = p10_range.get((y, m), p10_default)
    for i in p10:
        mapping[i] = "p10"

    # p11
    p11_start = max(p10) + 1
    for i in range(p11_start, 46):
        mapping[i] = "p11"

    return mapping



def show_problem_image_set(similar_problems, image_base="/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/수능 국어 AI 튜터링 시스템/언매/data/output_images/"):
    for i, p in enumerate(similar_problems, 1):
        year = p.get('year', '정보 없음')
        month = p.get('month', '정보 없음')
        qtext = p.get('문제', '') or ''
        number = extract_number_from_question(qtext)

        if year == '정보 없음' or month == '정보 없음' or not number:
            print(f"\n[{i}] ⚠️ 정보 없음: {year}-{month} / 번호: {number}")
            continue

        # 월을 항상 두 자리로 보정
        if month != '정보 없음':
            month_str = str(month).zfill(2)
        else:
            month_str = '정보 없음'

        if year == '정보 없음' or month_str == '정보 없음' or not number:
            print(f"\n[{i}] ⚠️ 정보 없음: {year}-{month_str} / 번호: {number}")
            continue

        number = int(number)
        mapping = get_passage_mapping(int(year), month_str)
        passage_key = mapping.get(number, None)

        print(f"\n--- 유사 문제 {i} ---")
        print(f"{year}-{month_str} / {number}번")

        # 지문 먼저 출력
        if passage_key:
            passage_img_path = os.path.join(image_base, f"{year}-{month_str}-언매_{passage_key}.png")
            print(passage_img_path)
            if os.path.exists(passage_img_path):
                print(f"[지문: {passage_key}]")
                Image.open(passage_img_path).show()
            else:
                print(f"[지문 {passage_key} 이미지 없음]")

        # 문제 이미지 출력
        problem_img_path = os.path.join(image_base, f"{year}-{month_str}-언매_{number}.png")
        print(problem_img_path)
        if os.path.exists(problem_img_path):
            print(f"[문제: {number}]")
            Image.open(problem_img_path).show()
        else:
            print(f"[문제 {number} 이미지 없음]")

In [ ]:
def pipeline_langmedia(context_image_path, question_image_path, top_k=3, image_base_dir=None):
    from PIL import Image

    # ✅ OCR로 지문/문제 텍스트 추출
    if context_image_path:
        passage = extract_text_with_underlines(context_image_path)
    else:
        passage = "(지문 없음)"

    question = extract_text_with_underlines(question_image_path)

    # ✅ GPT에게 정답 및 해설 요청
    gpt_input = text_prompt.format(passage=passage, question=question)
    gpt_output = ask_gpt(gpt_input)
    answer, explanation = parse_gpt_output(gpt_output)

    # ✅ GPT Vision으로 태깅 (문제 이미지 기반)
    external_problem = tag_from_image(question_image_path)
    external_problem["지문"] = passage  # 유사도 비교 위해 지문 추가

    # ✅ 유사 문제 추천
    similar_problems = recommend_for_external_problem(external_problem, top_n=top_k)

    # ✅ 이미지 경로 포함 (Streamlit 표시용)
    results = []
    for p in similar_problems:
        year = p.get('year', '')
        month = str(p.get('month', '')).zfill(2)
        qtext = p.get("문제", "")
        number = extract_number_from_question(qtext)

        result_entry = p.copy()
        result_entry["problem_number"] = number
        result_entry["passage_img_path"] = None
        result_entry["problem_img_path"] = None

        if image_base_dir and year and month and number:
            try:
                number = int(number)
                mapping = get_passage_mapping(int(year), month)
                passage_key = mapping.get(number, None)

                if passage_key:
                    result_entry["passage_img_path"] = os.path.join(image_base_dir, f"{year}-{month}-언매_{passage_key}.png")

                result_entry["problem_img_path"] = os.path.join(image_base_dir, f"{year}-{month}-언매_{number}.png")
            except:
                pass

        results.append(result_entry)

    # ✅ 최종 결과 리턴
    return {
        "answer": answer or "없음",
        "explanation": explanation or "없음",
        "similar_problems": results
    }